In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline

In [2]:
! cat ./data/HPRICE1.DES

In [3]:
data_path = "./data/"

header = ['price', 'assess', 'bdrms', 'lotsize', 'sqrft', 'colonial', 'lprice', 'lassess',
               'llotsize', 'lsqrft']

df_hprice1 = pd.read_table(data_path + "hprice1.raw", sep="\s+", header=None, names=header)

print('Shape =',df_hprice1.shape)
df_hprice1.head()

Shape = (88, 10)


,price,assess,bdrms,lotsize,sqrft,colonial,lprice,lassess,llotsize,lsqrft
0,300.0,349.1,4,6126,2438,1,5.703783,5.855359,8.720297,7.798934
1,370.0,351.5,3,9903,2076,1,5.913503,5.862210,9.200593,7.638198
2,191.0,217.7,3,5200,1374,0,5.252274,5.383118,8.556414,7.225482
3,195.0,231.8,3,4600,1448,1,5.273000,5.445875,8.433811,7.277938
4,373.0,319.1,4,6095,2514,1,5.921578,5.765504,8.715224,7.829630


# EXERCICE 1
# We consider the model :
> $price = \beta_0 + \beta_1 bdrms + \beta_2 lotsize + \beta_3 sqrft + u$

### On veut tester :
> $H_0 : Var(u|x_1,x_2, ···,x_k) = \sigma^2$

Since we assume that $u$ are zero mean, thus $Var(u|x) = \mathbf{E}(u^2|x)$ and : 
> $$H_0 : \mathbf{E}(u^2|x_1,x_2, ···,x_k) = \mathbf{E}(u^2) = \sigma^2$$
(c'est-à-dire que $u^2$ ne dépend pas des $x_1, ..., x_n$)

In [4]:
# Data
y = df_hprice1.price

X = df_hprice1[['bdrms', 'lotsize', 'sqrft']]
X.insert(0, 'cste', 1)  # loc, column_name, value
## OU
#X = np.concatenate((np.ones((len(y), 1)), df_hprice1[['bdrms', 'lotsize', 'sqrft']]), axis=1)

# Degrees of Freedom
n, k = X.shape
dof_ur = n - k
print('X shape :', X.shape)
print('dof_ur =', dof_ur)

## Régression Linéaire : Calculer les coeff beta
beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
# OU : en rajoutant '.values'
# y = y.values
# X = X.values
# beta = np.linalg.inv(X.T @ X) @ X.T @ y
print('beta =', beta)

# Résidus : u = y − X ∗ β
u = y - X.dot(beta)

## SSR_ur = somme des résidus au carrés du Unrestricted Model
#ssr_ur = u.T.dot(u)
#print('ssr_ur =', ssr_ur)

X shape : (88, 4)
dof_ur = 84
beta = [ -2.17703086e+01   1.38525219e+01   2.06770660e-03   1.22778185e-01]


### Same Calculations for the Restricted Model :
> Unrestricted model :  
$price = \beta_0 + \beta_1 bdrms + \beta_2 lotsize + \beta_3 sqrft + u$

> 

> Restricted model : all the coefficients except intercept are zeros  
==> $H_0 : \beta_1 = \beta_2 = \beta_3 = 0$ :  
$price = \beta_0 + u$

In [5]:
# Data
X_r = X[['cste']]

# Degrees of Freedom
n_r, k_r = X_r.shape
dof_r = n_r - k_r
print('X_r shape :', X_r.shape)
print('dof_r =', dof_r)

# On ne veut pas faire la régression des X par rapport aux y ici
# Mais bien les X par rapport aux u^2 (résidus du Unrestricted Model au carré)

#### Régression Linéaire : Calculer les coeff beta
##beta_r = np.linalg.inv(X_r.T.dot(X_r)).dot(X_r.T).dot(y)
##print('beta_r =', beta_r)
##
### Résidus : u = y − X ∗ β
##u_r = y - X_r.dot(beta_r)
##
### SSR_r = somme des résidus au carrés du Restricted Model
##ssr_r = u_r.T.dot(u_r)
##print('ssr_r =', ssr_r)

X_r shape : (88, 1)
dof_r = 87


## Mais ici, on veut calculer les SSR par rapport à $u^2$ et non par rapport à $y$ :

In [6]:
u2 = u ** 2

In [7]:
### UNRESTRICTED MODEL

## Régression Linéaire
beta_u2_ur = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(u2)
print('beta_u2_ur =', beta_u2_ur)

# Résidus : u = u^2 − X ∗ β_u2_ur
u = u2 - X.dot(beta_u2_ur)

# SSR_ur = somme des résidus au carrés du Unrestricted Model
ssr_u2_ur = u.T.dot(u)
print('ssr_u2_ur =', ssr_u2_ur)

beta_u2_ur = [ -5.52279479e+03   1.04176022e+03   2.01520923e-01   1.69103698e+00]
ssr_u2_ur = 3677520669.93


In [8]:
### RESTRICTED MODEL

## Régression Linéaire
beta_u2_r = np.linalg.inv(X_r.T.dot(X_r)).dot(X_r.T).dot(u2)
print('beta_u2_r =', beta_u2_r)

# Résidus : u = u^2 − X ∗ β_u2_r
u = u2 - X_r.dot(beta_u2_r)

# SSR_ur = somme des résidus au carrés du Unrestricted Model
ssr_u2_r = u.T.dot(u)
print('ssr_u2_r =', ssr_u2_r)

beta_u2_r = [ 3417.31598245]
ssr_u2_r = 4378734467.45


### Calcul de F :
$$F = \frac{(SSR\_u2\_r − SSR\_u2\_ur) / q}{SSR\_u2\_ur/(n−k−1)}$$

> avec $q = numerator \ \ degrees \ \ of \ \ freedom = dof\_r − dof\_ur$

> et (n−k−1) = degree of freedom (for the unrestricted model)

In [9]:
q = dof_r - dof_ur
print("q =", q)

F = ((ssr_u2_r - ssr_u2_ur) / q) / (ssr_u2_ur / dof_ur)
print("F =", F)

q = 3
F = 5.33891936786


### Calcul de la p-value :

In [10]:
p_value = scipy.stats.f.sf(F, q, dof_ur)
print("p_value =", p_value)

p_value = 0.00204774440966


### Interprétation  :

> ### p-valeur (source: wikipedia)  
Ce nombre est utilisé en statistiques inférentielles pour conclure sur le résultat d’un test statistique. La procédure généralement employée consiste à comparer la valeur-p à un seuil préalablement défini (traditionnellement 5 %). Si la valeur-p est inférieure à ce seuil, on rejette l'hypothèse nulle en faveur de l’hypothèse alternative, et le résultat du test est déclaré « statistiquement significatif »1. Dans le cas contraire, si la valeur-p est supérieure au seuil, on ne rejette pas l’hypothèse nulle, et on ne peut rien conclure quant aux hypothèses formulées.

> ## Ici, p-value < 0.05 donc on rejette $H_0$

> ### C'est-à-dire que le modèle n'est pas homoscédastique !

# EXERCICE 2
# We now consider the model with log(price):
> $log(price) = \beta_0 + \beta_1 bdrms + \beta_2 lotsize + \beta_3 sqrft + u$

In [11]:
### We perform the regression :

# Data
log_y = np.log(y)

# Régression Linéaire : Calculer les coeff beta
beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(log_y)
print('beta =', beta)

# Résidus : u = log(y) − X ∗ β
u = log_y - X.dot(beta)

beta = [  4.75937533e+00   2.52388099e-02   5.60178501e-06   3.64117400e-04]


In [12]:
### Compute u^2
u2 = u ** 2

### We consider :
> $u^2 = \delta_0 + \delta_1 bdrms + \delta_2 lotsize + \delta_3 sqrft + v$

### We want to test :
> $H_0 : \delta_1 = \delta_2 = \delta_3 = 0$

### Exercice : Compute F-statistic and p-value

In [13]:
### UNRESTRICTED MODEL

# Régression Linéaire
delta_u2_ur = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(u2)
print('delta_u2_ur =', delta_u2_ur)

# Résidus : u = u^2 − X ∗ β_u2_ur
u = u2 - X.dot(delta_u2_ur)

# SSR_ur = somme des résidus au carrés du Unrestricted Model
ssr_u2_ur = u.T.dot(u)
print('ssr_u2_ur =', ssr_u2_ur)

delta_u2_ur = [  1.64576832e-02   1.96929687e-02   8.50411453e-08  -2.63589574e-05]
ssr_u2_ur = 0.493377757572


In [14]:
### RESTRICTED MODEL

# Régression Linéaire
delta_u2_r = np.linalg.inv(X_r.T.dot(X_r)).dot(X_r.T).dot(u2)
print('delta_u2_r =', delta_u2_r)

# Résidus : u = u^2 − X ∗ β_u2_r
u = u2 - X_r.dot(delta_u2_r)

# SSR_ur = somme des résidus au carrés du Unrestricted Model
ssr_u2_r = u.T.dot(u)
print('ssr_u2_r =', ssr_u2_r)

delta_u2_r = [ 0.03441398]
ssr_u2_r = 0.514073082132


In [15]:
### CALCUL DE F
q = dof_r - dof_ur

F = ((ssr_u2_r - ssr_u2_ur) / q) / (ssr_u2_ur / dof_ur)
print("F =", F)

F = 1.1744937399


In [16]:
p_value = scipy.stats.f.sf(F, q, dof_ur)
print("p_value =", p_value)

p_value = 0.324417066143


## Interprétation  :

> ### Ici, p-value > 0.05 donc on ne rejette pas $H_0$

> #### On en conclut que le modèle est homoscédastique.

# EXERCICE 3 : WLS (weighted least squares)

Assume that we know the form of the heteroskedasticity $Var(u|x) = \sigma^2h(x)$  
with $h(x) = lotsize$ : the variance of the error is proportional to the size of the terrain

We consider the model :
> $$ \frac{price}{\sqrt{lotsize}} = \frac{\beta_0}{\sqrt{lotsize}}
+ \frac{\beta_1 bdrms}{\sqrt{lotsize}}
+ \frac{\beta_2 lotsize}{\sqrt{lotsize}}
+ \frac{\beta_3 sqrft}{\sqrt{lotsize}}
+ \frac{u}{\sqrt{lotsize}}$$

##### Verify that the above model satisfies the hypothesis of homoskedasticity.

In [17]:
# Refactor the data
sqrt_lotsize = np.sqrt(df_hprice1.lotsize)

y_wls = y / sqrt_lotsize

cste = (np.ones(len(y_wls)) / sqrt_lotsize).values.reshape((len(y), -1))
x1_wls = (df_hprice1.bdrms / sqrt_lotsize).values.reshape((len(y), -1))
x2_wls = (df_hprice1.lotsize / sqrt_lotsize).values.reshape((len(y), -1))
x3_wls = (df_hprice1.sqrft / sqrt_lotsize).values.reshape((len(y), -1))
X_wls = np.concatenate((cste, x1_wls, x2_wls, x3_wls), axis=1)

In [18]:
# Régression Linéaire : Calculer les coeff beta
beta = np.linalg.inv(X_wls.T.dot(X_wls)).dot(X_wls.T).dot(y_wls)
print('beta =', beta)

# Résidus : u = y_wls − X_wls ∗ β
u = y_wls - X_wls.dot(beta)

beta = [  1.17887557e+01   1.12359371e+01   5.51883227e-03   9.52907321e-02]


In [19]:
### Compute u^2
u2 = u ** 2

In [20]:
### UNRESTRICTED MODEL

# Régression Linéaire
beta_u2_ur = np.linalg.inv(X_wls.T.dot(X_wls)).dot(X_wls.T).dot(u2)
print('beta_u2_ur =', beta_u2_ur)

# Résidus : u = u^2 − X_wls ∗ β_u2_ur
u = u2 - X_wls.dot(beta_u2_ur)

# SSR_ur = somme des résidus au carrés du Unrestricted Model
ssr_u2_ur = u.T.dot(u)
print('ssr_u2_ur =', ssr_u2_ur)

beta_u2_ur = [  1.85860690e+01   1.13891590e+01   3.17779287e-03  -2.56000929e-02]
ssr_u2_ur = 70.184241756


In [21]:
### RESTRICTED MODEL

X_wls_r = np.reshape(X_wls[:, 0], (len(y), -1))

# Régression Linéaire
beta_u2_r = np.linalg.inv(X_wls_r.T.dot(X_wls_r)).dot(X_wls_r.T).dot(u2)
print('beta_u2_r =', beta_u2_r)

# Résidus : u = u^2 − X ∗ β_u2_r
u = u2 - X_wls_r.dot(beta_u2_r)

# SSR_ur = somme des résidus au carrés du Unrestricted Model
ssr_u2_r = u.T.dot(u)
print('ssr_u2_r =', ssr_u2_r)

beta_u2_r = [ 28.444881]
ssr_u2_r = 74.6220701815


In [22]:
### CALCUL DE F
q = dof_r - dof_ur

F = ((ssr_u2_r - ssr_u2_ur) / q) / (ssr_u2_ur / dof_ur)
print("F =", F)

F = 1.77047144494


In [23]:
p_value = scipy.stats.f.sf(F, q, dof_ur)
print("p_value =", p_value)

p_value = 0.159040529914


### Interprétation  :

> ### Ici, p-value > 0.05 donc on ne rejette pas $H_0$

> #### On en conclut que le modèle est là aussi homoscédastique.

# EXERCICE 4

...  
  
...

In [22]:
# sig2 = u′ ∗ u / (n−k)
sig2 = u.T.dot(u) / dof
print('sig2 =', sig2)

# std dv = sqrt(diag(sig2 ∗ inv(X′ ∗ X)))
std_dv = np.sqrt(np.diag(sig2 * np.linalg.inv(X.T.dot(X))))
print('std_dv =', std_dv)

sig2 = 0.194359331249
std_dv = [ 0.10419038  0.00732992  0.00172328  0.00309365]


In [ ]:
# obtenir en python la valeur critique à 95% pour une T distribution de dof degrees of freedom :
n_samples, n_features = X.shape
dof = n_samples - n_features
critic_val = scipy.stats.t.ppf(0.95, dof)